# LangChain + Youtube
**2024 삼성전자 무선사업부 생성형 AI 교육**

1. 기본 패키지 설치
2. Azure 계정 연결
3. Youtube 영상 불러오기
4. LLM 생성
5. 데이터 Split
6. Chain 연결


* 유사한 서비스 Lilys.ai [https://lilys.ai](https://lilys.ai)

In [ ]:
# 1. 패키지 설치

%pip install pytube
%pip install youtube-transcript-api

In [ ]:
# 2. 기본 패키지 설정 & AzureOpenAI 환경설정

# 환경 변수 설정!! 
import os
import langchain

from config_azure import (
    AZURE_OPENAI_API_VERSION,
    AZURE_OPENAI_ENDPOINT,
    AZURE_OPENAI_KEY
)
# lagchain 사용시 반드시 환경변수 값 사용
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_KEY
os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT
os.environ["OPENAI_API_VERSION"] = AZURE_OPENAI_API_VERSION

In [ ]:
# 3. Youtube 영상 데이터 읽어오기

from langchain.document_loaders import YoutubeLoader

loader = YoutubeLoader.from_youtube_url("")

data = loader.load()

In [ ]:
# 4. Azure OpenAI LLM(Chat Model 연결)

from langchain_openai import AzureChatOpenAI

chat_llm = AzureChatOpenAI(
    deployment_name="gpt-35-turbo",
    max_tokens=512,
    temperature=0.2,
)

In [ ]:
# 토큰 수 확인
chat_llm.get_num_tokens(data[0].page_content)

In [ ]:
# 5. Token Split

from langchain.text_splitter import TokenTextSplitter

splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=50)
docs = splitter.split_documents(data)

In [ ]:
# 6. Summarize Chain without LCEL

from langchain.chains.summarize import load_summarize_chain

chain = load_summarize_chain(
    llm=chat_llm,
    chain_type="map_reduce",
    return_intermediate_steps=True
)

chain.invoke({"input_documents" : docs}, return_only_outputs=True)

In [ ]:
# 7. Custom Prompt without LCEL

from langchain import PromptTemplate
from langchain.chains.summarize import load_summarize_chain

prompt_template = """{text}
# 유튜브 부분 영상 자막을 한글로 요약 정리한다
# 가독성을 높이기 위해 핵심 문장을 3~5개 리스트로 정리한다
# 예시
* 이 영상에서는 xx을 설명한다
* xx이 yy되었음을 zz를 통해 알 수 있다
"""

combine_prompt_template = """{text}
# 여러 개의 자막 요약본을 종합하여 영상의 전체 내용을 한글로 요약한다
"""

# map : 부분 영상 자막 각각 요약
MAP_PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])
# reduce : 부분 영상 요약본 총합
COMBINE_PROMPT = PromptTemplate(template=combine_prompt_template, input_variables=["text"])

chain = load_summarize_chain(
    llm=chat_llm,
    chain_type="map_reduce",
    return_intermediate_steps=True,
    map_prompt=MAP_PROMPT, combine_prompt=COMBINE_PROMPT)


In [ ]:
chain.invoke({"input_documents": docs}, return_only_outputs=True)

In [ ]:
# 8. Summarization Chain with LCEL

from langchain_core.output_parsers import StrOutputParser

# Map Chain
map_chain= (
    MAP_PROMPT
   | chat_llm
   | StrOutputParser() )

In [ ]:
final_summary = ""

for doc in docs:
   new_summary = map_chain.invoke({"text": doc.page_content})
   final_summary+=new_summary
   print("========================")
   print(new_summary)

In [ ]:
# 통합 Reduce Chain
reduce_chain= (
    COMBINE_PROMPT
   | chat_llm
   | StrOutputParser() )

# 스트림 활용하기
for chunk in reduce_chain.stream(final_summary):
    print(chunk, end="", flush=True) 